In [1]:
import spacy
import pandas as pd

# read in RecipeNLG Dataset

df = pd.read_csv('RecipeNLG_Dataset.csv')

In [2]:
# takeout relevant columns
# convert from string to list literal

from ast import literal_eval

df_cols = df[["ingredients","NER","directions"]]

data = df_cols.sample(n = 10000, random_state = 42)

data["ingredients"] = data["ingredients"].apply(literal_eval)  
data["directions"] = data["directions"].apply(literal_eval)
data["NER"] = data["NER"].apply(literal_eval)
data.reset_index(inplace=True, drop = True)


import spacy
nlp=spacy.load('en_core_web_sm')
from spacy.tokens import Doc
from spacy.training import Example

In [285]:
# data ingredients
import numpy as np
data_ingredients = data[["ingredients","NER"]]

def ingredients_to_string(x : list):
    return ", ".join(x)

def get_NER_labels(row):
    ingredients_sentence = row["ingredients"]

    sentence_indexed = np.array([0] * len(ingredients_sentence))

    NER = row["NER"]
    entity_list = []
    doc = nlp.make_doc(ingredients_sentence)
    for food in NER:
        try:
            food_start_index = ingredients_sentence.index(food)
            entity_list.append((food_start_index, food_start_index + len(food), "FOOD"))
        except ValueError:
            pass
    
    try:
        return Example.from_dict(doc, {"entities": entity_list})
    except ValueError:
        return Example.from_dict(doc, {"entities": []})

data_ingredients["ingredients"] = data_ingredients["ingredients"].apply(ingredients_to_string)

ingredients_NER_labels = data_ingredients.apply(get_NER_labels, axis = 1)

C:\Users\randymi\AppData\Local\Temp\ipykernel_19752\821735151.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ingredients["ingredients"] = data_ingredients["ingredients"].apply(ingredients_to_string)


In [110]:
import nltk
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

nltk.download('averaged_perceptron_tagger')

def remove_descriptor_nltk(food_name):
    words = nltk.word_tokenize(food_name)
    tagged_words = nltk.pos_tag(words)
    result_words = []
    for word, tag in tagged_words:
        if tag not in ['JJ', 'JJR', 'JJS']:
            result_words.append(word)
    return " ".join(result_words)
    
wnl.lemmatize(remove_descriptor_nltk("green onions"))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\randymi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [206]:
# data directions

data_directions = data[["directions","NER"]]

def directions_to_string(x : list):
    return " ".join(x)

def get_NER_labels(row):
    ingredients_sentence = row["directions"]

    sentence_indexed = np.array([0] * len(ingredients_sentence))

    NER = row["NER"]
    entity_list = []
    doc = nlp.make_doc(ingredients_sentence)
    for food in NER:
        try:
            food_start_index = ingredients_sentence.index(food)
            entity_list.append((food_start_index, food_start_index + len(food), "FOOD"))
        except ValueError:
            pass
    
    try:
        return Example.from_dict(doc, {"entities": entity_list})
    except ValueError:
        return Example.from_dict(doc, {"entities": []})

data_directions["directions"] = data_directions["directions"].apply(directions_to_string)

directions_NER_labels = data_directions.apply(get_NER_labels, axis = 1)

# filter out examples with no NER labels
directions_NER_labels = directions_NER_labels[directions_NER_labels.apply(lambda x: len(set(x.get_aligned_ner())) != 1)]

directions_NER_labels.reset_index(inplace=True, drop = True)



C:\Users\randymi\AppData\Local\Temp\ipykernel_19752\1479435369.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_directions["directions"] = data_directions["directions"].apply(directions_to_string)


In [286]:
# Getting the pipeline component and adding food label
ner = nlp.get_pipe("ner")
ner.add_label("FOOD")

def make_training_set(ingredients_proportion, directions_proportion, count = 1000):
    ingredients = ingredients_NER_labels.sample(n = int(count * ingredients_proportion))
    directions = directions_NER_labels.sample(n = int(count * directions_proportion))
    return pd.concat([ingredients, directions]).reset_index(drop = True)

# 80/20
training_set1 = make_training_set(0.8, 0.2, count = 1000)

# 60/40
training_set2 = make_training_set(0.6, 0.4, count = 1000)

# 50/50
training_set3 = make_training_set(0.5, 0.5, count = 1000)

# 40/60
training_set4 = make_training_set(0.4, 0.6, count = 1000)

# 20/80
training_set5 = make_training_set(0.2, 0.8, count = 1000)

TRAIN_DATA = ingredients_NER_labels


In [287]:
# training

optimizer = nlp.create_optimizer()
optimizer.learn_rate = 0.01


# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

def compound(start, stop, factor):

    # initialize counter
    value = start
    counter = 1

    # loop until counter is less than n
    while value < stop:

        # produce the current value of the counter
         
        yield value
        value = value * (factor ** counter)

        # increment the counter
        counter += 1
      

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 100 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4, 32, 1.05))    

    for batch in batches:
      nlp.update(batch, losses=losses, drop=0.5, sgd = optimizer)
      print(f"Losses {losses}")
    
    print(f"Epoch {iteration} Loses", losses)

    # every 10 epochs, save the model
    if iteration % 10 == 0:
      nlp.to_disk(f'tmp/ingredients_model{iteration}')
    


Losses {'ner': 86.4336348631884}
Losses {'ner': 130.26978915609172}
Losses {'ner': 153.78058697175277}
Losses {'ner': 201.68869669776234}
Losses {'ner': 243.96291910059574}
Losses {'ner': 330.32660221908066}
Losses {'ner': 378.88171184895384}
Losses {'ner': 420.2673018094507}
Losses {'ner': 442.85249833483306}
Losses {'ner': 505.3892123467273}
Losses {'ner': 560.4732225090489}
Losses {'ner': 627.976754576776}
Losses {'ner': 682.4428584949061}
Losses {'ner': 820.6637065656547}
Losses {'ner': 885.2210857675194}
Losses {'ner': 965.213486928322}
Losses {'ner': 1054.3949233992646}
Losses {'ner': 1181.9550326396459}
Losses {'ner': 1281.3015525398116}
Losses {'ner': 1363.4066647741313}
Losses {'ner': 1464.919168080862}
Losses {'ner': 1551.3965739952675}
Losses {'ner': 1639.406841696544}
Losses {'ner': 1760.096330518228}
Losses {'ner': 1903.7369983069123}
Losses {'ner': 1971.0473257976748}
Losses {'ner': 2093.905006881872}
Losses {'ner': 2247.3716716074664}
Losses {'ner': 2396.6784592026193}
L

In [2]:
# test each epoch on testing set
nlp = spacy.load('tmp/ingredients_model30')

In [13]:
nlp = spacy.load('tmp/ingredients_model20')
d = nlp("chicken, rice, and black beans, onions, garlic, salt, pepper, olive oil, and chicken broth")
d.ents

(chicken, rice, onions, garlic, salt, pepper, olive oil)

In [213]:
minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)).__next__()

[{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'B-FOOD', 'L-FOOD', 'O', 'O', 'O', 'O', 'O', 'U-FOOD', 'O', 'O', 'O', 'O', 'U-FOOD', 'O', 'O', 'O', 'B-FOOD', 'L-FOOD', 'O', 'O', 'O', 'B-FOOD', 'L-FOOD', 'O', 'O', 'O', 'O', 'B-FOOD', 'L-FOOD', 'O', 'O', 'O', 'O', 'U-FOOD', 'O', 'O', 'O', 'O', 'U-FOOD', 'O', 'O', 'O', 'B-FOOD', 'L-FOOD', 'O', 'O', 'O', 'U-FOOD'], 'spans': {}, 'links': {}}, 'token_annotation': {'ORTH': ['1', '1/2', 'lb', '.', 'ground', 'beef', ',', '1', '1/4', 'tsp', '.', 'onion', ',', '1', 'tsp', '.', 'parsley', ',', '1', 'can', 'mushroom', 'soup', ',', '1', 'c.', 'sour', 'cream', ',', '1/4', 'tsp', '.', 'garlic', 'powder', ',', '1', 'tsp', '.', 'salt', ',', '1/4', 'tsp', '.', 'pepper', ',', '1', 'can', 'tomato', 'soup', ',', '1/2', 'c.', 'milk'], 'SPACY': [True, True, False, True, True, False, True, True, True, False, True, False, True, True, False, True, False, True, True, True, True, False, True, True, True, True, False, True, True, False, True, True

In [160]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path
random.shuffle(TRAIN_DATA)
losses = {}

# batch up the examples using spaCy's minibatch
# minibatch is generator which returns a random sample of train_data of size = n. Compounding is generator returning 4 * (1.001)^i while < 32    
batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))

In [181]:
x = [[1,2,3]]
zip(*x)